In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device = "cuda" if torch.cuda.is_available() else "cpu"

class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

# Generator 모델
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Generator, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.batch_norm = nn.BatchNorm1d(hidden_size)

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.hidden_dropout = nn.Dropout(p=0.5)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        rbfs = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]

        hid = torch.zeros((rbfs.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(rbfs.size(1)):

            r = rbfs[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]

            gamma_r = self.temp_decay_r(d)
            gamma_h = self.temp_decay_h(d)

            hid = hid * gamma_h
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(r - x_hat) * m) / (torch.sum(m) + 1e-5)

            r_c = m * r + (1 - m) * x_hat

            r_hat = self.z_layer(r_c)

            x_loss += torch.sum(torch.abs(r - r_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_r, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * r_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(r - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * r + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            c_hat_list.append(c_hat.unsqueeze(1))
            imputations.append(c_c.unsqueeze(1))
            
            # GRU cell
            self.hidden_dropout(hid)
            hid = self.grucell(gru_input, hid)
            
        imputations = torch.cat(imputations, dim = 1)
        c_hat_list = torch.cat(c_hat_list, dim = 1)

        return  c_hat_list, imputations, x_loss / rbfs.size(1)


# Discriminator 모델
class Discriminator(nn.Module):
    def __init__(self, dim, h_dim):
        super(Discriminator, self).__init__()
        self.D_W1 = nn.Parameter(torch.Tensor(dim*2, h_dim))
        self.D_b1 = nn.Parameter(torch.zeros(h_dim))
        self.D_W2 = nn.Parameter(torch.Tensor(h_dim, h_dim))
        self.D_b2 = nn.Parameter(torch.zeros(h_dim))
        self.D_W3 = nn.Parameter(torch.Tensor(h_dim, dim))
        self.D_b3 = nn.Parameter(torch.zeros(dim))

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.D_W1.size(0))
        self.D_W1.data.uniform_(-stdv, stdv)
        self.D_b1.data.uniform_(-stdv, stdv)

        stdv_DW2 = 1. / math.sqrt(self.D_W2.size(0))
        self.D_W2.data.uniform_(-stdv_DW2, stdv_DW2)
        self.D_b2.data.uniform_(-stdv_DW2, stdv_DW2)

        stdv_DW3 = 1. / math.sqrt(self.D_W3.size(0))
        self.D_W3.data.uniform_(-stdv_DW3, stdv_DW3)
        self.D_b3.data.uniform_(-stdv_DW3, stdv_DW3)

    def forward(self, x, h):
        # Concatenate Data and Hint
        inputs = torch.cat((x, h), dim=2)
        D_h1 = torch.relu(torch.matmul(inputs, self.D_W1) + self.D_b1)
        D_h2 = torch.relu(torch.matmul(D_h1, self.D_W2) + self.D_b2)
        D_logit = torch.matmul(D_h2, self.D_W3) + self.D_b3
        D_prob = torch.sigmoid(D_logit)
        return D_prob

In [2]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)

def binary_sampler(p, rows, cols):
    unif_random_matrix = np.random.uniform(0., 1., size = [rows, cols])
    binary_random_matrix = 1. * (unif_random_matrix < p)

    return binary_random_matrix

def hint_matrix(B, Masks):
    Hint = B * Masks + 0.5 * (1-B)

    Hint = torch.from_numpy(Hint).to(torch.float32)
    return Hint
    
def missing_data_rbf(df,rbf, batch_size, p):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    binary_random_matrix = binary_sampler(p, shp[0], shp[1])

    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)
    Hint = hint_matrix(binary_random_matrix, masks)
    
    deltas = np.array(make_deltas(masks))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([rbf_x.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), Hint.unsqueeze_(0)], dim = 0)

    mydata  = MyDataset(dataset, batch_size)
    data = DataLoader(mydata, batch_size, shuffle=True)

    return data

In [3]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = 'air_1000_0.05_time.csv'
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size, 0.9)

In [4]:
def train(dataset, Generator, Discriminator, lr, epochs, alpha):
    imputation_list = []

    optimizer_G = optim.Adam(Generator.parameters(), lr=lr)
    optimizer_D = optim.Adam(Discriminator.parameters(), lr=lr)

    progress = tqdm(range(epochs))
    Generator.to(device)
    Discriminator.to(device)

    for epoch in progress:
        batch_loss_G = 0.0
        batch_loss_D = 0.0

        for data in dataset:

            data = data.to(device)
            hint = data[:,3,::].clone().detach()
            Mask = data[:,2,::].clone().detach()

            c_hat_list, imputations, x_loss = Generator(data)
            D_prob = Discriminator(imputations, hint)
            D_loss = -torch.mean(Mask * torch.log(D_prob + 1e-8) + (1-Mask) * torch.log(1. - D_prob + 1e-8))

            optimizer_D.zero_grad()
            D_loss.backward(retain_graph=True)
            optimizer_D.step()

            D_prob2 = Discriminator(imputations, hint)
            G_loss = -torch.mean((1-Mask) * torch.log(D_prob2 + 1e-8)) + x_loss * alpha
            optimizer_G.zero_grad()
            G_loss.backward(retain_graph=True)
            optimizer_G.step()

            imputation_list.append(imputations)
            batch_loss_G += G_loss
            batch_loss_D += D_loss

        progress.set_description("G_loss: {}, D_loss : {}".format(batch_loss_G, batch_loss_D))
    
    return imputation_list

In [5]:
G = Generator(36, 64)
D = Discriminator(36, 64)

In [6]:
imputation_list = train(dataset, G, D, 1e-3, 300, 10)

G_loss: 17.324520111083984, D_loss : 0.28490614891052246: 100%|██████████| 300/300 [05:23<00:00,  1.08s/it]


In [7]:
def val_missing_data_rbf(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([rbf_x.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf(df, rbf)
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    c_hat_list, imputations, x_loss = model(dataset)

    Nonscale_imputataion = pd.DataFrame(imputations[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    rbf_df = pd.DataFrame(rbf_df.values, columns= df.columns)
    rbf_df = (rbf_df * df.std()) + df.mean()
    
    real = real.fillna(0)
    df = df.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(imputations[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(imputations[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print("RBF MAE :", np.sum(np.abs((rbf_df - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("RBF MRE :", np.sum(np.abs((rbf_df - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    print('observation MAE :', np.sum(np.abs((Nonscale_imputataion - df).values * masks.cpu().numpy())) / np.sum(masks.cpu().numpy()))

    return Nonscale_imputataion

In [8]:
realpath = 'pm25_ground.txt'
real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)

In [9]:
# imputation
eval_model(G, rbfpath, 'pm25_ground.txt', dfpath)

Scale MAE : tensor(0.2640)
Scale MRE : tensor(0.3811)
Original MAE : 21.31256209026835
Original MRE : 0.29937425816287694
RBF MAE : 26.244345089085357
RBF MRE : 0.36865024996703916
observation MAE : 8.382986230262004


,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,112.979632,94.558028,109.012972,105.292300,104.623389,98.806747,102.341168,99.229637,100.945096,96.546109,...,76.712119,78.509137,93.531377,75.436973,89.525562,74.103350,88.368280,109.708148,97.749255,120.733266
1,116.773533,97.222771,112.847540,109.052867,106.300672,101.935345,106.501526,101.818824,104.554178,98.539958,...,91.890872,80.232474,96.495995,87.667646,93.647284,76.142415,91.739778,111.762013,104.154877,128.011903
2,119.546584,98.990375,116.139787,113.003751,108.182353,105.169956,110.835437,105.005373,108.455424,101.542161,...,98.136107,81.931966,98.443932,90.834839,97.075322,78.074774,95.394895,114.699009,105.383608,129.231866
3,121.463774,99.790112,119.029576,117.346926,110.447253,108.663489,115.472482,109.059993,112.779486,105.811113,...,99.612019,83.787046,99.414389,92.096992,99.833949,80.079464,99.463593,118.629706,107.473838,133.860040
4,112.106772,99.871445,121.933568,122.414836,113.389459,112.696303,120.620953,114.270940,117.706829,111.515048,...,104.045623,86.126502,99.783051,93.022497,102.235314,82.504918,104.119098,123.537302,107.458295,134.063385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,85.517552,66.028307,70.094838,65.534252,73.493956,66.055605,64.616278,64.528036,70.011654,69.110241,...,90.454438,99.399387,84.389473,76.380683,86.552459,119.705787,111.664071,96.144254,76.555851,94.116831
8755,82.107537,66.425545,73.556263,67.037746,74.498008,65.691271,72.138699,64.140210,70.108171,70.659920,...,97.684677,109.433097,77.498082,74.398222,80.346946,129.726792,125.189406,125.314947,75.624343,86.125582
8756,81.051560,69.216705,79.531669,70.265830,76.753433,67.533268,83.520990,65.965787,72.694449,74.851661,...,109.155502,122.580940,74.110290,75.558825,78.026563,139.256283,140.241324,160.064574,77.533807,82.975097
8757,87.653015,66.563618,85.776118,71.924839,84.343692,70.316736,95.181907,92.213810,76.202058,93.339675,...,112.299218,123.555588,79.823029,78.028144,65.881984,104.524212,132.610261,131.240331,81.575055,88.246833


In [13]:
# C_hat
eval_model(G, rbfpath, 'pm25_ground.txt', dfpath)

Scale MAE : tensor(0.2604)
Scale MRE : tensor(0.3760)
Original MAE : 21.00848766572539
Original MRE : 0.2951029718253502
RBF MAE : 26.244345089085357
RBF MRE : 0.36865024996703916
observation MAE : 8.382986230262004


,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,112.979632,94.558028,109.012972,105.292300,104.623389,98.806747,102.341168,99.229637,100.945096,96.546109,...,79.074474,78.509137,93.531377,72.966515,89.525562,74.103350,88.368280,109.708148,105.481484,123.406619
1,116.773533,97.222771,112.847540,109.052867,106.300672,101.935345,106.501526,101.818824,104.554178,98.539958,...,91.561790,80.232474,96.495995,92.394345,93.647284,76.142415,91.739778,111.762013,105.008387,134.513446
2,119.546584,98.990375,116.139787,113.003751,108.182353,105.169956,110.835437,105.005373,108.455424,101.542161,...,96.995659,81.931966,98.443932,89.657708,97.075322,78.074774,95.394895,114.699009,110.871501,138.300555
3,121.463774,99.790112,119.029576,117.346926,110.447253,108.663489,115.472482,109.059993,112.779486,105.811113,...,99.452154,83.787046,99.414389,88.090290,99.833949,80.079464,99.463593,118.629706,116.924954,143.958600
4,113.827661,99.871445,121.933568,122.414836,113.389459,112.696303,120.620953,114.270940,117.706829,111.515048,...,104.048894,86.126502,99.783051,89.740898,102.235314,82.504918,104.119098,123.537302,123.434968,150.469390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,85.517552,66.028307,70.094838,65.534252,73.493956,66.055605,64.616278,64.528036,70.011654,69.110241,...,102.305976,102.483719,84.389473,76.380683,86.552459,119.705787,111.664071,96.144254,76.555851,104.854518
8755,82.107537,66.425545,73.556263,67.037746,74.498008,65.691271,72.138699,64.140210,70.108171,70.659920,...,110.105233,113.897149,77.498082,74.398222,80.346946,129.726792,125.189406,125.314947,75.624343,99.625134
8756,81.051560,69.216705,79.531669,70.265830,76.753433,67.533268,83.520990,65.965787,72.694449,74.851661,...,122.394518,129.151693,74.110290,75.558825,78.026563,139.256283,140.241324,160.064574,77.533807,96.273548
8757,98.486822,78.911588,85.776118,90.013126,91.644665,70.316736,95.181907,93.776060,76.202058,91.275036,...,122.006243,129.238375,81.155703,78.028144,85.374034,113.069793,113.175050,126.007220,105.082755,103.485884
